#### 导入相关包

In [1]:
import sentence_transformers
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from typing import List
from PowerMarketQA.CustomLLM import CustomLLM

#### 加载外部知识

In [3]:
import pandas as pd
import os

# 假设Excel文件都在一个名为'data'的文件夹下
folder_path = './all_data/excel'
excel_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.xlsx')]

data_frames = []
for file in excel_files:
    df = pd.read_excel(file)
    data_frames.append(df)

combined_data = pd.concat(data_frames, ignore_index=True)
combined_data

,question,answer
0,1、什么是光伏发电？什么是分布式光伏系统？,光伏发电是指利用太阳能光伏电池把太阳辐射能直接转变成电能的发电方式。光伏发电是当今太阳能...
1,2、什么是户用分布式光伏系统？,户用分布式光伏系统是指利用自然人宅基地范围内的建筑物，比如自有住宅，以及附属物建设的分布...
2,3、您知道光伏发电的历史起源吗？,1839 年，19 岁的法国贝克勒尔做物理实验时，发现在导电液中的两种金属电极用光照射时...
3,4、光伏电池是怎么发电的？,光伏电池是一种具有光－电转换特性的半导体器件，它直接将太阳辐射能转换成直流电，是光伏发电...
4,5、户用分布式光伏发电系统由哪些部件构成？,户用分布式光伏发电系统由光伏方阵（光伏方阵由光伏组件串并联而成)、光伏逆变器、光伏支架、...
...,...,...
407,169.什么是电力系统的运行操作？,电力系统中的设备有运行、热备用、冷备用和检修四种不同的状态，运行操作是指变更电力系统设备...
408,170.倒闸操作的分类有哪些？有哪些要求？,倒闸操作从满足新技术应用和经济性，即减人增效提高劳动生产率的要求来考虑，倒闸操作可分为就...
409,171.什么是遥控操作和程序操作？有哪些要求？,遥控操作是指从调度端或集控站发出远方操作指令，以微机监控系统或变电站的RTU当地功能为技...
410,172.倒闸操作的基本要求是什么？包括哪些内容？,倒闸操作是一项严谨的工作，应根据值班调度员或运行值班负责人的指令，在受令人复诵无误后执行...


In [6]:
questions = combined_data['question'].tolist()
answers = combined_data['answer'].tolist()
QA_list = []
for q,a  in zip(questions,answers):
    qa = q + '\n' + a
    QA_list.append(qa)

print(QA_list[0])

# 将文本块封装成Document类
docs = []
for i in range(0,len(QA_list)):
    qa_doc = Document(page_content=QA_list[i], metadata={"source": "电力知识","order":i+1})
    docs.append(qa_doc)

print(docs[0])

1、什么是光伏发电？什么是分布式光伏系统？
  光伏发电是指利用太阳能光伏电池把太阳辐射能直接转变成电能的发电方式。光伏发电是当今太阳能发电的主流。
  分布式光伏系统是指在用户场地附近建设，一般接入低于 35kV 及以下电压等级的电网，所发电以就地消纳为主，且在配电系统平衡调节为特性的光伏发电设施。分布式光伏系统的运行方式自发自用、余电上网，全额上网两种方式。自
发自用、余电上网是指分布式光伏发电系统所发电力由电力用户优先使用，多余电量接入电网；全额上网是指分布式光伏发电系统所发电力全部接入电网。
  分布式发电遵循因地制宜、清洁高效、分散布局、就近利用的原则，充分利用当地的太阳能资源，替代和减少化石能源消费。
page_content='1、什么是光伏发电？什么是分布式光伏系统？
  光伏发电是指利用太阳能光伏电池把太阳辐射能直接转变成电能的发电方式。光伏发电是当今太阳能发电的主流。
  分布式光伏系统是指在用户场地附近建设，一般接入低于 35kV 及以下电压等级的电网，所发电以就地消纳为主，且在配电系统平衡调节为特性的光伏发电设施。分布式光伏系统的运行方式自发自用、余电上网，全额上网两种方式。自
发自用、余电上网是指分布式光伏发电系统所发电力由电力用户优先使用，多余电量接入电网；全额上网是指分布式光伏发电系统所发电力全部接入电网。
  分布式发电遵循因地制宜、清洁高效、分散布局、就近利用的原则，充分利用当地的太阳能资源，替代和减少化石能源消费。' metadata={'source': '电力知识', 'order': 1}


#### 将相关知识存储到向量数据库中

In [7]:
# 从huggingface上加载embedding模型，对文本块向量化
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="moka-ai/m3e-base",  # 使用m3e模型做embeddding
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
# 使用FAISS向量数据库存储
vectorstore = FAISS.from_documents(docs, huggingface_embeddings)
# 构建索引
retriever = vectorstore.as_retriever()

D:\Download\miniconda3\envs\qwen_1.8B\lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
D:\Download\miniconda3\envs\qwen_1.8B\lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


#### 根据query检索相关文本

In [8]:
query = """电力市场与普通商品市场有哪些差异？"""  # Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search(query,k=2)
for i, doc in enumerate(relevant_documents):
    print(f"检索到的第{i+1}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")


检索到的第1个内容: 
 2.电力市场与普通商品市场有哪些差异？
  电力市场相较于普通商品市场具有显著的特殊性，归根到底就在于电力商品与其他一般商品的差异性。电力商品本质即电能，与其他商品最本质的差异在于其自然属性和社会属性。
（1）电力商品具有无仓储性。电能的生产、交割和消费几乎是同时完成的，其交割速度远快于一般商品，因此也不存在一般商品一手交钱一手交货的交易方式。
（2）电力商品具有同质性。电能不带有任何生产者的标识，电能生产者将生产的电能输入电网，即完成了生产过程；而电能的使用者也只能从电网获取所需数量的电能，电能生产者和消费者间可以达成交易，但在电能实际生产和消费过
程中不存在对应性。
（3）电力商品具有可预测性。电能需求在较长周期内会以日或周为单位呈现周期性波动，一定程度上抑制了投机行为，但也增加了市场主体滥用市场力的可能性。
（4）电力商品具有生产资料和生活资料的双重属性，因此它既关系国计，又关系民生。电力市场既是生产资料市场，又是生活资料市场，还是十分典型的无仓储公共市场。
-----------------------------------------------------
检索到的第2个内容: 
 1.什么是电力市场？
  电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现电力商品交换的电力工业组织结构、经营管理和运行规则的总和。电力市场又是一个具体的执行系统，包括交易场所、交易管理系统、计量和结算系
统、信息和通信系统等。上面从组织和实操两个维度对电力市场进行了描绘。通常电力市场包括广义和狭义两种含义：
（1）比照商品市场的一般定义，广义的电力市场泛指电力流通交换的领域。按照该含义，自电力作为商品实现交换之日起，电力市场就已经存在。当今在涉及市场规模和范围的语境下，运用的便是广义电力市场的概念。广义的电力市场
有着明确的地域和容量指向，电力市场的地理边界可能差异很大，例如单一州（省）域内的电力市场，或者整个国家的电力市场，乃至跨国的电力市场。而这种地理边界受限于电网的覆盖范围，由于电网是电力传输的唯一通道，故多大范
围的电网才可能有多大范围的电力市场；电力市场容量也同样存在很大的差异。
（2）狭义的电力市场是指现代竞争性的电力市场，《中国电力百科全书》定义的电力市场即为狭义的电力市场，建立电力市场旨在通过开放、竞争

#### 获取上下文信息

In [15]:
import json
with open('./all_data/json/test.json','r',encoding='UTF-8') as json_file:
	tests = json.load(json_file)
    
tests_with_doc = []
for test in tests:
    query = test['question']
    relevant_documents = vectorstore.similarity_search(query,k=2)
    relevant_documents = [doc.page_content for doc in relevant_documents]
    test['relevant_documents'] = relevant_documents
    tests_with_doc.append(test)
    
tests_with_doc[0]

{'question': '45.避雷器的性能有哪些？',
 'answer': '  在确定避雷器性能上，有以下几个重要关系：\n（1）从保证避雷器运行安全方面考虑，其持续运行电压应等于或大于电网的最大运行相电压。\n（2）从保证避雷器不会老化太快方面考虑，其额定电压应等于或大于电网持续运行电压的1.25倍。\n（3）避雷器在标称电流（雷电流）下的残压，等于额定电压（峰值）乘以保护比，目前避雷器的保护比约等于1.6~1.7。',
 'relevant_documents': ['45.避雷器的性能有哪些？\n  在确定避雷器性能上，有以下几个重要关系：\n（1）从保证避雷器运行安全方面考虑，其持续运行电压应等于或大于电网的最大运行相电压。\n（2）从保证避雷器不会老化太快方面考虑，其额定电压应等于或大于电网持续运行电压的1.25倍。\n（3）避雷器在标称电流（雷电流）下的残压，等于额定电压（峰值）乘以保护比，目前避雷器的保护比约等于1.6~1.7。',
  '43.什么是避雷器？\n  避雷器是一种释放过电压能量限制过电压幅值的保护设备，它通过并联放电间隙或非线性电阻的作用，对入侵流动波进行削幅，以降低被保护是所受过电压幅值。避雷器既可用来防护大气过电压，也可用来防护操作过电压，其性能是变\n电站其他设备绝缘水平的基础。']}

#### 导出上下文信息

In [16]:
import json

with open('./all_data/json/test.json', 'w', encoding='utf-8') as f:
    json.dump(tests_with_doc, f, ensure_ascii=False, indent=4)

#### 构建RAG流程链

In [32]:
# 构造prompt
def prompt_template(question):
    relevant_documents = vectorstore.similarity_search(question,k=2)
    context = ""
    for doc in relevant_documents:
        context += doc.page_content + "\n"
    prompt = "请使用以下检索到的背景信息来回答问题。如果你不知道答案，直接说你不知道。\n"  + f"背景：\n{context}\n" + f"问题：{question}\n\n" 
    return prompt


In [26]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

model_dir = "E:\pythonProject\Qwen\\finetune\output_qwen"

# 加载微调过的模型
model = AutoPeftModelForCausalLM.from_pretrained(
    f'{model_dir}',
    device_map="auto",
    trust_remote_code=True
).eval()

tokenizer = AutoTokenizer.from_pretrained(f'{model_dir}', revision='master', trust_remote_code=True)

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

D:\Download\miniconda3\envs\qwen_1.8B\lib\site-packages\peft\utils\save_and_load.py:224: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(filename

In [33]:
# 回复问题
prompt= prompt_template("电力市场与普通商品市场有哪些差异？")
response, history = model.chat(tokenizer, prompt , history=None)
print(response)

电力市场与普通商品市场的主要区别在于电力商品的本质及相关的生产要素不同，从市场需求角度分析电力商品与一般商品最大的不同之处在于其无仓储性以及生产与消费同步的特点。从社会属性角度看，电力作为一种天然能源不仅存在于商品交换之中，而是作为实物资源的一部分参与流通，这就决定了电力的价值不只体现在产量的增长中，还体现在物权的流转与配置之中。此外，电力商品不可再生性决定了电力商品的稀缺性，为保障电力供应，通常需对电量进行价格管制，以补偿利用电力的成本损失，从而维护电力系统的正常运作。以上均属电力商品本身的社会属性，为不同类型的电力商品提供了不同的价值认定基础。而普通商品是以货币计量的商品，与电子设备相关的所有技术交易行为，交易的基本对象均为价格，然而价格受供需双方行为影响较大，交易双方的真实成本无法得到体现。不同商品的互动决定了某些商品的市场价格难以用单一的通货膨胀率来描述，例如黄金因为其稀有性可明显地反映通货膨胀的程度。电力商品虽然以电价的形式直接与需求者打交道，但由于电力商品实物价值的获取是在电网之外通过竞价的方式完成，这一特性使电力商品的价格很难由某种固定的指数来衡量，如购买水电的价格就与时间的关系很大，只要出现供应短缺时，往往价格就会上升，这对建立一个完善的电力市场来说，是一个巨大的挑战。由于电力商品涉及到电网以外的部分供电基础设施和多种替代能源，没有石油等其他商品那样可以通过地理范围或者供应链的划分把不同地域的企业和个体组织排除在外。电力交易买卖的是服务和费用的总称，如何正确界定各种外部服务关系对于电力行业长期发展至关重要。


#### 在数据集上测试回复效果

In [30]:
questions[:3]

['1.什么是电力市场？', '2.电力市场与普通商品市场有哪些差异？', '3.电力市场有哪些特征？']

In [34]:
rag_results = []
for index,query in enumerate(questions):
    prompt = prompt_template(query)
    response, history = model.chat(tokenizer, prompt , history=None)
    rag_results.append({'query':query,
                    'query_id':index+1,
                    'reference':answers[index],
                    'response':response})

    print(f"第{index+1}个问题已回复..........")


第1个问题已回复..........
第2个问题已回复..........
第3个问题已回复..........
第4个问题已回复..........
第5个问题已回复..........
第6个问题已回复..........
第7个问题已回复..........
第8个问题已回复..........
第9个问题已回复..........
第10个问题已回复..........
第11个问题已回复..........
第12个问题已回复..........
第13个问题已回复..........
第14个问题已回复..........
第15个问题已回复..........
第16个问题已回复..........
第17个问题已回复..........
第18个问题已回复..........
第19个问题已回复..........
第20个问题已回复..........
第21个问题已回复..........
第22个问题已回复..........
第23个问题已回复..........
第24个问题已回复..........
第25个问题已回复..........
第26个问题已回复..........
第27个问题已回复..........
第28个问题已回复..........
第29个问题已回复..........
第30个问题已回复..........
第31个问题已回复..........
第32个问题已回复..........
第33个问题已回复..........
第34个问题已回复..........
第35个问题已回复..........
第36个问题已回复..........
第37个问题已回复..........
第38个问题已回复..........
第39个问题已回复..........
第40个问题已回复..........
第41个问题已回复..........
第42个问题已回复..........
第43个问题已回复..........
第44个问题已回复..........
第45个问题已回复..........
第46个问题已回复..........
第47个问题已回复..........
第48个问题已回复..........
第49个问题已回复..........
第50个问题已回复..........
第51个问题已回复

IndexError: list index out of range

In [38]:
prompt = prompt_template(questions[124])
response, history = model.chat(tokenizer, prompt , history=None)
rag_results.append({'query':query,
                    'query_id':125,
                    'reference':answers[124],
                    'response':response})

In [40]:
rag_results[124]

{'query': '125.电力现货技术支持系统与电力交易平台定位与关系是什么？',
 'query_id': 125,
 'reference': '随着中发9号文的发布，我国电力市场改革不断深化，电力交易品种日益丰富，以中长期交易规避风险，以现货市场发现价格的“中长期+现货”联合运营模式逐渐形成。中发9号文配套文件《关于推进电力市场建设的实施意见》指出，电力市场主要由中长期市场和现货市场构成，中长期市场主要开展多年、年、季、月、周等日以上电能量交易和可中断负荷、调压等辅助服务交易。现货市场主要开展日前、日内、实施电能量交易和备用、调频等辅助服务交易。条件成熟时，探索开展容量市场、电力期货和衍生品等交易。2018年，国家电网公司《全国统一电力市场深化设计方案》在综合考虑体制架构和电网安全运行需要的基础上，提出了由交易中心负责市场注册、交易申报、交易结算、信息发布等与市场交易密切相关环节，由调控中心负责现货和辅助服务交易组织、安全校核等与电网运行密切相关环节的统一市场联合运营模式。此模式也成为了电力市场技术支撑平台在交易中心和调控中心之间功能建设的实际划分标准，交易中心的电力交易平台、调控中心的电力现货技术支持系统各功能相互协作实现“全时段、全过程、全环节”的交易流程，实现“中长期+现货”市场安全、高效、有序运营，向市场成员提供准确规范的市场交易信息和在线交易业务支持。调控中心负责现货交易与电网运行密切相关的环节，负责安全校核、现货交易组织、辅助服务交易组织等工作。电力现货技术支持系统面向调控中心业务，用于支撑日前、实时等电力现货市场的组织、出清、安全校核等核心业务，并与调控中心内部的调度系统、交易中心的电力交易平台等系统实现对接。交易中心负责与市场交易密切相关的环节，电力交易平台作为国网公司对市场主体服务的窗口，在市场注册、交易申报、交易结算、信息发布等环节为市场主体提供相关服务，并与包括电力现货技术支持系统在内的调度、营销、财务等系统实现对接。电力交易平台分为面向电力交易中心的主站端，以及面向发电侧、用电侧等市场成员的子站端。主站端的主要功能是市场成员注册审批、中长期交易组织、政府授权合约分解、合同管理、结算计算、信息发布、信息管理、信用管理等功能；子站端是发用侧市场成员用于进行市场注册、现货市场申报、中长期市场申报、中长期合约备案、查看发布信息、查看结算账单

In [41]:
import json
with open("response/power_qa_rag_results_qwen_lora.json", "w", encoding='utf-8') as f:
   json.dump(rag_results, f, ensure_ascii=False, indent=4)